# Deep Learning for Natural Language Processing with Pytorch
This tutorial will walk you through the key ideas of deep learning programming using Pytorch.
Many of the concepts (such as the computation graph abstraction and autograd) are not unique to Pytorch and are relevant to any deep learning tool kit out there.

I am writing this tutorial to focus specifically on NLP for people who have never written code in any deep learning framework (e.g, TensorFlow, Theano, Keras, Dynet).

In [26]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# 1. Introduction to Torch's tensor library

All of deep learning is computations on tensors, which are generalizations of a matrix that can be indexed in more than 2 dimensions.  We will see exactly what this means in-depth later.  First, lets look what we can do with tensors.

### Creating Tensors
Tensors can be created from Python lists with the torch.Tensor() function.

In [8]:
# Create a torch.Tensor object with the given data.  It is a 1D vector
V_data = [1., 2., 3.]
V = torch.Tensor(V_data)
print x

# Creates a matrix
M_data = [[1., 2., 3.], [4., 5., 6]]
M = torch.Tensor(M_data)
print y

# Create a 3D tensor of size 2x2x2.
T_data = [[[1.,2.], [3.,4.]],
          [[5.,6.], [7.,8.]]]
T = torch.Tensor(T_data)
print z


 1
 2
 3
[torch.FloatTensor of size 3]


 1  2  3
 4  5  6
[torch.FloatTensor of size 2x3]


(0 ,.,.) = 
  1  2
  3  4

(1 ,.,.) = 
  5  6
  7  8
[torch.FloatTensor of size 2x2x2]



What is a 3D tensor anyway?
Think about it like this.
If you have a vector, indexing into the vector gives you a scalar.  If you have a matrix, indexing into the matrix gives you a vector.  If you have a 3D tensor, then indexing into the tensor gives you a matrix!

A note on terminology: when I say "tensor" in this tutorial, it refers to any torch.Tensor object.  Matrices and vectors are special cases of torch.Tensors, where their dimension is 1 and 2 respectively.  When I am talking about 3D tensors, I will explicitly use the term "3D tensor".

In [9]:
# Index into V and get a scalar
print V[0]

# Index into M and get a vector
print M[0]

# Index into T and get a matrix
print T[0]

1.0

 1
 2
 3
[torch.FloatTensor of size 3]


 1  2
 3  4
[torch.FloatTensor of size 2x2]



You can also create tensors of other datatypes.  The default, as you can see, is Float.
To create a tensor of integer types, try torch.LongTensor().  Check the documentation for more data types, but Float and Long will be the most common.

You can create a tensor with random data and the supplied dimensionality with torch.randn()

In [10]:
x = torch.randn((3, 4, 5))
print x


(0 ,.,.) = 
  0.1146 -0.6506  0.6839 -1.5281 -0.3123
 -0.3978  0.1204  1.0106  0.6426 -0.3773
  1.5972  0.6948 -0.6626 -0.5126  2.1116
  0.8082  2.5599 -0.5388 -0.2966 -0.8011

(1 ,.,.) = 
  0.4382  0.2241 -0.8473  0.2425  0.9720
 -0.3337  0.7472 -0.4915 -0.5668 -0.8585
 -2.1146  0.2108 -0.6764  0.0754  1.8036
  0.8029 -1.2479 -0.3994 -0.8097 -1.0615

(2 ,.,.) = 
  1.2940  0.3598  1.5550 -0.1175 -0.2281
  0.4670 -0.9777  1.7070  0.3330  2.5000
 -1.6602  0.1536  1.2721  1.3831 -0.1833
 -0.0308 -1.9315  0.4497  1.4138  0.5602
[torch.FloatTensor of size 3x4x5]



### Operations with Tensors
You can operate on tensors in the ways you would expect.

In [11]:
x = torch.Tensor([ 1., 2., 3. ])
y = torch.Tensor([ 4., 5., 6. ])
z = x + y
print z


 5
 7
 9
[torch.FloatTensor of size 3]



See [the documentation](http://pytorch.org/docs/torch.html) for a complete list of the massive number of operations available to you.  They expand beyond just mathematical operations.

One helpful operation that we will make use of later is concatenation.

In [22]:
# By default, it concatenates along the first axis (concatenates rows)
x_1 = torch.randn(2, 5)
y_1 = torch.randn(3, 5)
z_1 =torch.cat([x_1, y_1])
print z_1

# Concatenate columns:
x_2 = torch.randn(2, 3)
y_2 = torch.randn(2, 5)
z_2 = torch.cat([x_2, y_2], 1) # second arg specifies which axis to concat along
print z_2

# If your tensors are not compatible, torch will complain.  Uncomment to see the error
# torch.cat([x_1, x_2])


-0.0180  0.0584  0.9895  0.6425  1.8225
 1.3009  2.2458  0.0894  0.7866 -0.9593
-0.6731  0.5934 -0.6335 -0.4634  2.0126
 0.0406 -0.5549  0.5613 -2.4119 -0.2033
-0.3331 -0.2922 -1.7489 -0.4690 -0.7442
[torch.FloatTensor of size 5x5]


-0.5333  1.3768  0.8385  0.2887  0.0408  1.0201  0.9143  0.3634
 0.3216 -0.6005 -1.8512 -0.5111  0.7530  0.4623  0.7390 -1.6262
[torch.FloatTensor of size 2x8]



### Reshaping Tensors
Use the .view() method to reshape a tensor.
This method receives heavy use, because many neural network components expect their inputs to have a certain shape.
Often you will need to reshape before passing your data to the component.

In [28]:
x = torch.randn(2, 3, 4)
print x
print x.view((2, 12)) # Reshape to 2 rows, 12 columns
print x.view((2, -1)) # Same as above.  If one of the dimensions is -1, its size can be inferred


(0 ,.,.) = 
 -0.2645 -0.4485  0.2376 -1.4220
 -1.5924 -0.4594 -0.2210 -0.2319
 -1.1506 -0.2188 -1.4698 -1.8785

(1 ,.,.) = 
  0.9020 -0.1623 -1.5317 -0.1645
 -1.9074  0.2076 -1.1382  1.4981
 -0.0087 -0.0006  0.6136 -0.2514
[torch.FloatTensor of size 2x3x4]



Columns 0 to 9 
-0.2645 -0.4485  0.2376 -1.4220 -1.5924 -0.4594 -0.2210 -0.2319 -1.1506 -0.2188
 0.9020 -0.1623 -1.5317 -0.1645 -1.9074  0.2076 -1.1382  1.4981 -0.0087 -0.0006

Columns 10 to 11 
-1.4698 -1.8785
 0.6136 -0.2514
[torch.FloatTensor of size 2x12]



Columns 0 to 9 
-0.2645 -0.4485  0.2376 -1.4220 -1.5924 -0.4594 -0.2210 -0.2319 -1.1506 -0.2188
 0.9020 -0.1623 -1.5317 -0.1645 -1.9074  0.2076 -1.1382  1.4981 -0.0087 -0.0006

Columns 10 to 11 
-1.4698 -1.8785
 0.6136 -0.2514
[torch.FloatTensor of size 2x12]




# 2. Computation Graphs and Automatic Differentiation

The concept of a computation graph is essential to efficient deep learning programming, because it allows you to not have to write the back propogation gradients yourself.  A computation graph is simply a specification of how your data is combined to give you the output.  Since the graph totally specifies what parameters were involved with which operations, it contains enough information to compute derivatives.  This probably sounds vague, so lets see what is going on using the fundamental class of Pytorch: autograd.Variable.

First, think from a programmers perspective.  What is stored in the torch.Tensor objects we were creating above?
Obviously the data and the shape, and maybe a few other things.  But when we added two tensors together, we got an output tensor.  All this output tensor knows is its data and shape.  It has no idea that it was the sum of two other tensors (it could have been read in from a file, it could be the result of some other operation, etc.)

The Variable class keeps track of how it was created.  Lets see it in action.

In [49]:
# Variables wrap tensor objects
x = autograd.Variable( torch.Tensor([1., 2., 3]), requires_grad=True )
# You can access the data with the .data attribute
print x.data

# You can also do all the same operations you did with tensors with Variables.
y = autograd.Variable( torch.Tensor([4., 5., 6]), requires_grad=True )
z = x + y
print z.data

# BUT z knows something extra.
print z.creator


 1
 2
 3
[torch.FloatTensor of size 3]


 5
 7
 9
[torch.FloatTensor of size 3]



So Variables know what created them.  z knows that it wasn't read in from a file, it wasn't the result of a multiplication or exponential or whatever.  And if you keep following z.creator, you will find yourself at x and y.

But how does that help us compute a gradient?

In [50]:
# Lets sum up all the entries in z
s = z.sum()
print s
print s.creator

Variable containing:
 21
[torch.FloatTensor of size 1]



So now, what is the derivative of this sum with respect to the first component of x?  In math, we want
$$ \frac{\partial s}{\partial x_0} $$
Well, s knows that it was created as a sum of the tensor z.  z knows that it was the sum x + y.
So 
$$ s = \overbrace{x_0 + y_0}^\text{$z_0$} + \overbrace{x_1 + y_1}^\text{$z_1$} + \overbrace{x_2 + y_2}^\text{$z_2$} $$
And so s contains enough information to determine that the derivative we want is 1!

Of course this glosses over the challenge of how to actually compute that derivative.  The point here is that s is carrying along enough information that it is possible to compute it.  In reality, the developers of Pytorch program the sum() and + operations to know how to compute their gradients, and run the back propogation algorithm.  An in-depth discussion of that algorithm is beyond the scope of this tutorial.

Lets have Pytorch compute the gradient, and see that we were right: (note if you run this block multiple times, the gradient will increment.  That is because Pytorch *accumulates* the gradient into the .grad property, since for many models this is very convenient.)

In [51]:
s.backward() # calling .backward() on any variable will run backprop, starting from it.
print x.grad

Variable containing:
 1
 1
 1
[torch.FloatTensor of size 3]



Understanding what is going on in the block below is crucial for being a successful programmer in deep learning.

In [54]:
x = torch.randn((2,2))
y = torch.randn((2,2))
z = x + y # These are Tensor types, and backprop would not be possible

var_x = autograd.Variable( x )
var_y = autograd.Variable( y )
var_z = var_x + var_y # var_z contains enough information to compute gradients, as we saw above
print var_z.creator

var_z_data = var_z.data # Get the wrapped Tensor object out of var_z...
new_var_z = autograd.Variable( var_z_data ) # Re-wrap the tensor in a new variable

# ... does new_var_z have information to backprop to x and y?
# NO!
print new_var_z.creator
# And how could it?  We yanked the tensor out of var_z (that is what var_z.data is).  This tensor
# doesn't know anything about how it was computed.  We pass it into new_var_z, and this is all the information
# new_var_z gets.  If var_z_data doesn't know how it was computed, theres no way new_var_z will.
# In essence, we have broken the variable away from its past history

None


Here is the basic, extremely important rule for computing with autograd.Variables (note this is more general than Pytorch.  There is an equivalent object in every major deep learning toolkit):

** If you want the error from your loss function to backpropogate to a component of your network, you MUST NOT break the Variable chain from that component to your loss Variable.  If you do, the loss will have no idea your component exists, and its parameters can't be updated. **

I say this in bold, because this error can creep up on you in very subtle ways (I will show some such ways below), and it will not cause your code to crash or complain, so you must be careful.

# 3. Deep Learning Building Blocks: Affine maps, non-linearities and objectives

Deep learning consists of composing linearities with non-linearities in clever ways.  The introduction of non-linearities allows for powerful models.  In this section, we will play with these core components, make up an objective function, and see how the model is trained.

### Affine Maps
One of the core workhorses of deep learning is the affine map, which is a function $$f(x)$$ where
$$ f(x) = Ax + b $$ for a matrix $A$ and vectors $x, b$.  The parameters to be learned here are $A$ and $b$.  Often, $b$ is refered to as the *bias* term.

### Non-Linearities
First, note the following fact, which will explain why we need non-linearities in the first place.
Suppose we have two affine maps $f(x) = Ax + b$ and $g(x) = Cx + d$.  What is $f(g(x))$?
$$ f(g(x)) = A(Cx + d) + b = ACx + (Ad + b) $$
$AC$ is a matrix and $Ad + b$ is a vector, so we see that composing affine maps gives you an affine map.

From this, you can see that if you wanted your neural network to be long chains of affine compositions, that this adds no new power to your model than just doing a single affine map.

If we introduce non-linearities in between the affine layers, this is no longer the case, and we can build much more powerful models.

There are a few core non-linearities.  $\tanh(x), \sigma(x), \text{ReLU}(x)$ are the most common.
You are probably wondering: "why these functions?  I can think of plenty of other non-linearities."
The reason for this is that they have gradients that are easy to compute, and computing gradients is essential for learning.  For example
$$ \frac{d\sigma}{dx} = \sigma(x)(1 - \sigma(x)) $$

A quick note: although you may have learned some neural networks in your intro to AI class where $\sigma(x)$ was the default non-linearity, typically people shy away from it in practice.  This is because the gradient *vanishes* very quickly as the absolute value of the argument grows.  Small gradients means it is hard to learn.  Most people default to tanh or ReLU.

### Softmax and Probabilities
The function $\text{Softmax}(x)$ is also just a non-linearity, but it is special in that it usually is the last operation done in a network.  This is because it takes in a vector of real numbers and returns a probability distribution.  Its definition is as follows.  Let $x$ be a vector of real numbers (positive, negative, whatever, there are no constraints.  Then the i'th component of $\text{Softmax}(x)$ is
$$ \frac{\exp(x_i)}{\sum_i \exp(x_i)} $$
It should be clear that the output is a probability distribution: each element is non-negative and the sum over all components is 1.

You could also think of it as just applying an element-wise exponentiation operator to the input (to make everything non-negative) and then dividing by the normalization constant.

### Objective Functions
The objective function is the function that your network is being trained to minimize (in which case it is often called a *loss function* or *cost function*).
This proceeds by first choosing a training instance, running it through your neural network, and then computing the loss of the output.  The parameters of the model are then updated by taking the derivative of the loss function.

The idea behind minimizing the loss function on your training examples is that your network will hopefully generalize well and have small loss on unseen examples in your dev set, test set, or in production.
An example loss function is the *negative log*, which is a very common objective for multi-class classification.  For supervised multi-class classification, this means training the network to minimize the negative log probability of the correct output (or equivalently, maximize the log probability of the correct output).

# 4. Word Embeddings: Encoding Lexical Semantics

### An Example: N-Gram Language Modeling
Recall that in an n-gram language model, given a sequence of words $w$, we want to compute
$$ P(w_i | w_{i-1}, w_{i-2}, \dots, w_{i-n+1} ) $$
Where $w_i$ is the ith word of the sequence.

In this example, we will compute the loss function on some training examples.  We won't yet train the network.  We will expand on this example soon.

In [9]:
N = 3
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [ ([test_sentence[i], test_sentence[i+1]], test_sentence[i+2]) for i in xrange(len(test_sentence) - 2) ]
print trigrams[:3] # print the first 3, just so you can see what they look like

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [16]:
vocab = set(test_sentence)
word_to_ix = { word: i for i, word in enumerate(vocab) }

In [33]:
EMBED_DIM = 5

embeddings = nn.Embedding(len(vocab), EMBED_DIM)
linear = nn.Linear(2 * EMBED_DIM, len(vocab))
loss_function = nn.NLLLoss()

In [37]:
losses = []
for context, target in trigrams:
    context_idxs = map(lambda w: word_to_ix[w], context)
    context_var = autograd.Variable( torch.LongTensor(context_idxs) )
    context_embeds = embeddings(context_var).view(1, -1)
    predictions = F.log_softmax(linear(context_embeds))
    loss = loss_function(predictions, autograd.Variable(torch.LongTensor([word_to_ix[target]])))
    losses.append(loss)
print losses[:5]

[Variable containing:
 4.7115
[torch.FloatTensor of size 1]
, Variable containing:
 3.5177
[torch.FloatTensor of size 1]
, Variable containing:
 4.8611
[torch.FloatTensor of size 1]
, Variable containing:
 5.0375
[torch.FloatTensor of size 1]
, Variable containing:
 4.5100
[torch.FloatTensor of size 1]
]


# 5. Creating Network Components in Pytorch

# 6. Making Decisions

# 7. Sequence Models and Long-Short Term Memory Networks